In [15]:
import requests,json
import re
from bs4 import BeautifulSoup 

In [33]:
list_url = "https://y.qq.com/n/yqq/playlist/7628476949.html"
response = requests.get(list_url)
html_txt = response.text 
# 原本尝试抓json的，毕竟json处理起来easy，但是抓不到
# json的地址不是上面的那个url，抓到的json 显示 invalid refere
# 问了HFF，他说应该是 异步请求，防跨域的
# 所以打算直接 正则匹配，提取 歌名 - 歌手
html_txt

'<!DOCTYPE html>\r\n<html lang="zh-cn">\r\n<head>\r\n    <meta charset="utf-8" />\r\n    <meta http-equiv="X-UA-Compatible" content="IE=Edge" />\r\n    <meta http-equiv="Content-Language" content="zh-cn" />\r\n    <meta name="Copyright" content="Tencent" />\r\n    <!--关键字和描述一定要写-->\r\n    <meta name="description" content="" />\r\n    <meta name="keywords" content="" />\r\n    <!--如果该页面只适合在电脑上进行浏览，请添加以下代码-->\r\n    <meta name="applicable-device" content="pc">\r\n    <title>404 - QQ音乐-千万正版音乐海量无损曲库新歌热歌天天畅听的高品质音乐平台！</title>\r\n    <style>\r\n        body,h2,p{margin:0;padding:0;}\r\n        body{font-family:\\5FAE\\8F6F\\96C5\\9ED1,Tahoma,Arial,sans-serif;line-height:2;color:#666;}\r\n        .none_txt__tit{padding-top:120px;font-weight:normal;font-size:40px;color:#333;}\r\n        .none_txt{text-align:center;font-size:20px;}\r\n        a{color:#31c27c;text-decoration:none;margin:0 4px;}\r\n    </style>\r\n\r\n</head>\r\n<script>\r\nwindow.clickgotohome = function(){\r\n\twindow.location.r

In [47]:
list_url = "https://y.qq.com/n/yqq/playlist/7628476949.html"
headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'
    }
response = requests.get(list_url, headers=headers)
html_txt = response.text 
html_txt

n            <div class="footer_product">\r\n                <h3 class="footer_tit">特色产品</h3>\r\n                <ul class="footer_product_list">\r\n                    <li class="footer_product_list__item footer_product_list__item--pic"><a href="http://kg.qq.com/?pgv_ref=qqmusic.y.topmenu" rel="noopener" target="_blank" class="js_other_link" data-stat="y_new.footer.other_music.k"><i class="icon_qm_kg"></i>全民K歌</a></li>\r\n                    <li class="footer_product_list__item footer_product_list__item--pic"><a href="http://y.qq.com/vip/voice_intro/index.html" rel="noopener" target="_blank" class="js_other_link" data-stat="y_new.footer.other_music.supersound"><i class="icon_qm_ss"></i>Super Sound</a></li>\r\n                    <li class="footer_product_list__item footer_product_list__item--pic"><a href="http://y.qq.com/y/static/down/qplay.html?pgv_ref=qqmusic.y.topmenu" rel="noopener" target="_blank" class="js_other_link" data-stat="y_new.footer.other_music.qplay"><i class="icon_qm_

In [48]:
soup = BeautifulSoup(html_txt, 'html.parser')
soup.select('.songlist__item')
soup.find_all('div', class_='songlist__item')

[<div class="songlist__item">
 <div class="songlist__edit songlist__edit--check">
 <input class="songlist__checkbox" type="checkbox"/>
 </div>
 <div class="songlist__number">1</div>
 <div class="songlist__songname">
 <span class="songlist__songname_txt"><a href="javascript:;" title="Youth (Acoustic)">Youth (Acoustic)</a></span>
 <div class="mod_list_menu">
 <a class="list_menu__item list_menu__play" href="javascript:;" title="播放">
 <i class="list_menu__icon_play"></i>
 <span class="icon_txt">播放</span>
 </a>
 <a aria-haspopup="true" class="list_menu__item list_menu__add" data-target="menu_add" href="javascript:;" title="添加到歌单">
 <i class="list_menu__icon_add"></i>
 <span class="icon_txt">添加到歌单</span>
 </a>
 <a aria-haspopup="true" class="list_menu__item list_menu__down" data-target="menu_down" href="javascript:;" title="下载">
 <i class="list_menu__icon_down"></i>
 <span class="icon_txt">下载</span>
 </a>
 <a aria-haspopup="true" class="list_menu__item list_menu__share" data-aria="menu_shar